In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import re
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from tensorflow import keras
# load appropriate tokenizer and fine-tuned model
tokenizer = AutoTokenizer.from_pretrained('facebook/esm2_t33_650M_UR50D', do_lower_case=False,trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("facebook/esm2_t33_650M_UR50D")

with open('test.csv') as l:
  with open('pred.txt','w') as r:
    with open('prob.txt','w') as f:
      lines=l.readlines()
      for line in lines:
        input_seq=line.split(',')[1]
        input_seq_spaced = ' '.join([input_seq[i:i+1] for i in range(0, len(input_seq), 1) ])
        input_seq_spaced = re.sub(r'[UZOB]', 'X', input_seq_spaced)
        input_seq_tok = tokenizer(input_seq_spaced, return_tensors = 'pt')
        output = model(**input_seq_tok)
        logits = output[0]
        y_prob = torch.sigmoid(logits)[:,1].detach().numpy()
        f.write(str(y_prob)[1:-1])
        f.write('\n')
        y_pred = y_prob > 0.5
        if y_pred == True:
          input_class ='1'
        else:
          input_class ='0'
        r.write(input_class)
        r.write('\n')


In [ ]:

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
y_test=pd.read_csv('test.csv',header=None)
pre=pd.read_csv('pred.txt',header=None)
prod=pd.read_csv('prob.txt',header=None)

# Evaluate the model
accuracy = accuracy_score(y_test, pre)
f1 = f1_score(y_test, pre)
precision = precision_score(y_test, pre)
MCC = matthews_corrcoef(y_test, pre)
auc = roc_auc_score(y_test, prod)
conf_matrix = confusion_matrix(y_test, pre)

TP = conf_matrix[0][0]
FN = conf_matrix[0][1]
TN = conf_matrix[1][1]
FP = conf_matrix[1][0]
Sn = TP / (TP + FN)
Sp = TN / (TN + FP)

# Print the evaluation metrics
print(f'Sensitivity: {Sn:.4f}')
print(f'Specificity: {Sp:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test Matthews Correlation Coefficient: {MCC:.4f}')
print(f'Test ROC AUC Score: {auc:.4f}')
      